In [1]:
#statistical
import pandas as pd
import numpy as np

#dataset
from sklearn.datasets import load_iris

#mchine learning
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#graphing and display
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

#feature engineer
import feature_engineer as fe

In [2]:
iris = load_iris(return_X_y = True)
iris_x = pd.DataFrame(iris[0], columns = ['col_' + str(i) for i in range(iris[0].shape[1])])
iris_y = iris[1]

In [3]:
logistic_clf = LogisticRegression()

In [4]:
eng = fe.FeatureEngineer(train_test_split, logistic_clf, f1_score, n_best = 15)
eng_test = eng.fit_transform(iris_x, iris_y)

C:\Users\rossr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [5]:
eng_test.head()

,col_0,col_1,col_2,col_3,min_(col_0)_truediv_col_3,max_(col_2)_sub_col_2,col_3_sub_col_1,min_(col_1)_truediv_col_3,min_(col_0)_sub_col_3,col_2_sub_col_1,col_0_truediv_col_2,col_2_sub_col_0,min_(col_2)_truediv_col_3,col_3_mul_col_2,col_3_mul_col_0,max_(col_2)_sub_col_0,col_2_mul_col_1,col_2_add_col_1,max_(col_0)_truediv_col_0
0,5.1,3.5,1.4,0.2,21.5,5.5,-3.3,10.0,4.1,-2.1,3.642857,-3.7,5.0,0.28,1.02,1.8,4.90,4.9,1.549020
1,4.9,3.0,1.4,0.2,21.5,5.5,-2.8,10.0,4.1,-1.6,3.500000,-3.5,5.0,0.28,0.98,2.0,4.20,4.4,1.612245
2,4.7,3.2,1.3,0.2,21.5,5.6,-3.0,10.0,4.1,-1.9,3.615385,-3.4,5.0,0.26,0.94,2.2,4.16,4.5,1.680851
3,4.6,3.1,1.5,0.2,21.5,5.4,-2.9,10.0,4.1,-1.6,3.066667,-3.1,5.0,0.30,0.92,2.3,4.65,4.6,1.717391
4,5.0,3.6,1.4,0.2,21.5,5.5,-3.4,10.0,4.1,-2.2,3.571429,-3.6,5.0,0.28,1.00,1.9,5.04,5.0,1.580000


In [6]:
eng_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 19 columns):
col_0                        150 non-null float64
col_1                        150 non-null float64
col_2                        150 non-null float64
col_3                        150 non-null float64
min_(col_0)_truediv_col_3    150 non-null float64
max_(col_2)_sub_col_2        150 non-null float64
col_3_sub_col_1              150 non-null float64
min_(col_1)_truediv_col_3    150 non-null float64
min_(col_0)_sub_col_3        150 non-null float64
col_2_sub_col_1              150 non-null float64
col_0_truediv_col_2          150 non-null float64
col_2_sub_col_0              150 non-null float64
min_(col_2)_truediv_col_3    150 non-null float64
col_3_mul_col_2              150 non-null float64
col_3_mul_col_0              150 non-null float64
max_(col_2)_sub_col_0        150 non-null float64
col_2_mul_col_1              150 non-null float64
col_2_add_col_1              150 non-null flo

In [7]:
X_train, X_test, y_train, y_test = train_test_split(eng_test, iris_y, test_size = 0.3, random_state = 42)
logistic_clf = LogisticRegression()
logistic_clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
y_pred = logistic_clf.predict(X_test)
y_pred_proba = logistic_clf.predict_proba(X_test)

In [9]:
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))
print('Precision: {:.2f}'.format(precision_score(y_test, y_pred, average = 'weighted')))
print('Recall: {:.2f}'.format(recall_score(y_test, y_pred, average = 'weighted')))
print('F1 score: {:.2f}'.format(f1_score(y_test, y_pred, average = 'weighted')))

Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1 score: 1.00


In [14]:
for i, s in enumerate(sorted(eng.best_cols, reverse = True)):
    if i < 14:
        print(i, s, eng.best_cols[s].columns)

0 1.0 Index(['min_(col_0)_truediv_col_3'], dtype='object')
1 0.977744855967 Index(['max_(col_2)_sub_col_2'], dtype='object')
2 0.955555555556 Index(['col_3_sub_col_1'], dtype='object')
3 0.955291005291 Index(['min_(col_1)_truediv_col_3'], dtype='object')
4 0.930857709119 Index(['min_(col_0)_sub_col_3'], dtype='object')
5 0.908956228956 Index(['col_2_sub_col_1'], dtype='object')
6 0.884621949138 Index(['col_0_truediv_col_2'], dtype='object')
7 0.864729344729 Index(['col_2_sub_col_0'], dtype='object')
8 0.859166666667 Index(['min_(col_2)_truediv_col_3'], dtype='object')
9 0.832287790183 Index(['col_3_mul_col_2'], dtype='object')
10 0.803631082062 Index(['col_3_mul_col_0'], dtype='object')
11 0.773007856341 Index(['max_(col_2)_sub_col_0'], dtype='object')
12 0.772773109244 Index(['col_2_mul_col_1'], dtype='object')
13 0.753809047142 Index(['col_2_add_col_1'], dtype='object')
